In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
# map 32^3  linear indices to 4^3 linear  indices
# map 32^3 linear -> 32^3 coords -> 4^3 coords -> 4^3 linear

In [10]:
import torch
from datasets.scannet.utils_3d import ProjectionHelper

In [4]:
in_dims = (32, 32, 32)
out_dims = (4, 4, 4)

In [59]:
lin_inds = torch.arange(0, in_dims[0]*in_dims[1]*in_dims[2], out=torch.LongTensor())
coords = lin_inds.new_empty(4, lin_inds.size(0))

In [60]:
lin_inds[:20]

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])

In [61]:
lin_inds.shape, coords.shape

(torch.Size([32768]), torch.Size([4, 32768]))

In [62]:
coords = ProjectionHelper.lin_ind_to_coords_static(lin_inds, coords, in_dims)

print(coords[:, :10])
print(coords[:, 1000:1010])

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
        [31, 31, 31, 31, 31, 31, 31, 31, 31, 31],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1]])


In [65]:
factor = torch.LongTensor(in_dims) // torch.LongTensor(out_dims)
factor

tensor([8, 8, 8])

In [66]:
new_coords = coords.clone()
new_coords.shape
print(new_coords[:, :10])

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [69]:
# check a few output coords
print((new_coords[:3, :10].T // factor).T)
print((new_coords[:3, 1000:1010].T // factor).T)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 2, 2],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [70]:
new_coords[:3, :] = (new_coords[:3, :].T // factor).T

In [71]:
new_coords[:3, -10:]

tensor([[2, 2, 3, 3, 3, 3, 3, 3, 3, 3],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]])

In [72]:
# convert coords back to lin inds
# dims are L,B,H
# Z*(L*B) + Y*L + X

tensor(32768)

In [80]:
ProjectionHelper.coords_to_lin_inds(coords, torch.prod(torch.LongTensor(in_dims)), in_dims)

tensor([32768,     0,     1,  ..., 32765, 32766, 32767])

In [83]:
out_inds = ProjectionHelper.coords_to_lin_inds(new_coords, torch.prod(torch.LongTensor(in_dims)), out_dims)
out_inds[:20]

tensor([32768,     0,     0,     0,     0,     0,     0,     0,     0,     1,
            1,     1,     1,     1,     1,     1,     1,     2,     2,     2])